In [1]:
import numpy as np
import pandas as pd
from skimage import measure
import anndata
import xarray

In [2]:
def get_cell_ids(coordinates, masks):
    #Coords falling at edge get rounded down
    coordinates = np.where(coordinates>=np.array(masks.shape)-0.5,np.floor(coordinates),np.round(coordinates)).astype(np.uint16)

    labels = []
    for row, col in coordinates:
        label = masks[row,col]

        labels.append(label)
            
    return pd.Series(labels)

In [3]:
tile = 'tile40'
SEG_FILE = f'/media/gambino/students_workdir/ilias/colonCancerPatient2_tiles/gt_masks/{tile}.nc'
DETECTED_TRANSCRIPTS = f'/media/gambino/students_workdir/ilias/data/images/{tile}/transcripts/tile_40_detected_pixel_coords.csv'

In [4]:
dataset = xarray.load_dataset(SEG_FILE)

masks = dataset.get('masks').values
images = dataset.get('images').values
z_stack = 3

In [5]:
transcripts = pd.read_csv(DETECTED_TRANSCRIPTS)

In [9]:
cells_detected = len(np.unique(masks[3]))
cells_detected

12250

In [7]:
#assign transcripts to cells
transcripts['mask_label'] = get_cell_ids(transcripts[['local_row','local_col']].to_numpy(),masks[z_stack])
transcripts.head(3)

,barcode_id,global_x,global_y,global_z,x,y,fov,gene,transcript_id,local_row,local_col,mask_label
0,0,90316.979482,20046.289615,0.0,837.8820,1266.4530,342,PDK4,ENST00000005178,46.289615,316.979482,0
1,0,91008.439735,20055.425760,0.0,1529.3359,1275.5897,342,PDK4,ENST00000005178,55.425760,1008.439735,0
2,0,91150.505896,20274.885855,0.0,1671.4036,1495.0476,342,PDK4,ENST00000005178,274.885855,1150.505896,0


In [10]:
#filter Blanks
transcripts_filtered = transcripts[~transcripts.loc[:,'transcript_id'].str.contains('Blank')]

In [11]:
unassigned_transcripts = np.sum(transcripts_filtered.loc[:,'mask_label']==0)
percentage_assigned_transcripts = (1 - unassigned_transcripts/len(transcripts_filtered)) * 100
percentage_assigned_transcripts

68.82354011467471

In [38]:
#filter unassigned transcripts
transcripts_filtered = transcripts_filtered[~transcripts_filtered.loc[:,'mask_label'].eq(0)]
transcripts_filtered

,barcode_id,global_x,global_y,global_z,x,y,fov,gene,transcript_id,local_row,local_col,mask_label
4,0,91424.314045,20376.837169,0.0,1945.20750,1597.00000,342,PDK4,ENST00000005178,376.837169,1424.314045,615855
5,0,91387.109989,20427.552177,0.0,1908.00000,1647.71420,342,PDK4,ENST00000005178,427.552177,1387.109989,615855
10,0,90708.103557,20561.531391,0.0,1229.00000,1781.69260,342,PDK4,ENST00000005178,561.531391,708.103557,615868
12,0,90333.100005,20719.421807,0.0,854.00000,1939.58250,342,PDK4,ENST00000005178,719.421807,333.100005,616900
15,0,90416.434128,20469.504189,1.0,937.33420,1689.66580,342,PDK4,ENST00000005178,469.504189,416.434128,615860
...,...,...,...,...,...,...,...,...,...,...,...,...
9497488,499,99736.948336,29978.669094,3.0,769.67510,123.76685,800,NOTCH1,ENST00000651671,9978.669094,9736.948336,530273
9497490,499,99768.263447,29982.453368,3.0,801.00000,127.55102,800,NOTCH1,ENST00000651671,9982.453368,9768.263447,530273
9497498,499,99721.735229,29971.175547,5.0,754.46670,116.27135,800,NOTCH1,ENST00000651671,9971.175547,9721.735229,530273
9497505,499,99743.837290,29968.882946,6.0,776.56976,113.98012,800,NOTCH1,ENST00000651671,9968.882946,9743.837290,530273


In [39]:
transcripts_by_cell = transcripts_filtered.groupby(['mask_label','gene'])['transcript_id'].count()
count_matrix = transcripts_by_cell.unstack()
count_matrix = count_matrix.fillna(0)
count_matrix

gene,ACKR3,ACTA2,ADAMTS4,AKT1,AKT2,AKT3,AMOTL2,ANGPT1,ANGPT2,APC,...,WNT3A,WNT5A,WWTR1,XBP1,XCL1,XCR1,YAP1,ZAP70,ZBED2,ZEB1
mask_label,,,,,,,,,,,,,,,,,,,,,
7596,1.0,0.0,1.0,6.0,0.0,2.0,0.0,0.0,3.0,1.0,...,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0
7597,0.0,1.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,1.0,5.0,0.0,0.0,1.0,0.0,8.0,0.0
7598,2.0,1.0,0.0,5.0,1.0,0.0,6.0,1.0,0.0,1.0,...,1.0,0.0,1.0,2.0,0.0,0.0,13.0,1.0,1.0,1.0
7599,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,1.0,0.0
7600,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,3.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
705140,2.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0
705141,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
705142,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0


In [40]:
region_props = measure.regionprops_table(masks[z_stack],properties=('label','bbox','centroid','area','axis_major_length','axis_minor_length'))
bbox_array = np.array([region_props[f'bbox-{i}'] for i in range(4)]).swapaxes(0,1)
cell_ids = [f'{str(label)}_' + '_'.join(coord.astype(np.str_))
            for label, coord in zip(region_props['label'], bbox_array)]
bbox_str_list = [','.join(coord.astype(np.str_)) for label,coord in zip(region_props['label'],bbox_array)]

obs = pd.DataFrame({'cell_id':cell_ids,
                    'mask_label':region_props['label'],
                    'bbox':bbox_str_list,
                    'centroid-0':region_props['centroid-0'],
                    'centroid-1':region_props['centroid-1'],
                    'axis_major_length':region_props['axis_major_length'],
                    'axis_minor_length':region_props['axis_minor_length'],
                    'area':region_props['area']
                    })

obs = obs.set_index('cell_id')

#filter cells by those in count matrix and assign them to obs field
obs = obs[obs.loc[:,'mask_label'].isin(count_matrix.index)]

count_matrix.sort_index(inplace=True)
obs.sort_values(by='mask_label',inplace=True)
count_matrix.index = obs.index

In [41]:
count_matrix_ann = anndata.AnnData(count_matrix,dtype=np.uint32)
count_matrix_ann.obs = obs
count_matrix_ann.obs

,mask_label,bbox,centroid-0,centroid-1,axis_major_length,axis_minor_length,area
cell_id,,,,,,,
7596_0_2010_93_2113,7596,"0,2010,93,2113",45.507658,2057.688285,109.680729,85.044677,7247.0
7597_37_3330_93_3396,7597,"37,3330,93,3396",61.828131,3364.045538,68.568696,51.853015,2723.0
7598_37_3450_167_3552,7598,"37,3450,167,3552",108.106342,3506.736172,131.445293,91.276359,8877.0
7599_111_4659_167_4751,7599,"111,4659,167,4751",137.323662,4699.303434,93.839303,52.468402,3757.0
7600_157_4871_214_4937,7600,"157,4871,214,4937",183.948817,4900.310585,69.310639,50.791298,2579.0
...,...,...,...,...,...,...,...
705140_8122_6052_8197_6127,705140,"8122,6052,8197,6127",8155.671742,6087.641292,77.904241,71.367987,4335.0
705141_8131_4410_8188_4448,705141,"8131,4410,8188,4448",8159.277710,4430.278928,60.143913,35.322795,1642.0
705142_8150_4972_8215_5038,705142,"8150,4972,8215,5038",8179.948357,5002.281690,75.216194,53.344501,2982.0


In [42]:
count_matrix_ann.uns.update(dict(transcript_coordinates = transcripts_filtered,
                                 image = dataset.images[z_stack].values,
                                 masks = dataset.masks[z_stack].values,
                                 img_files = dataset.attrs['files']))

In [44]:
count_matrix_ann.write(f'/media/gambino/students_workdir/ilias/data/images/tile40/transcripts/tile_40_gt_z{z_stack}_expression_matrix.h5ad')